**ASSIGNMENT 4**

Group Name: Group 81
Member Names: Ishan Phadte and Eeshaan Patel
Member Student Numbers: 300238878 and 300165634
Report Title: CSI 4106 A4

**Derived Datasets**

This notebook is a starting point for Assignment 4. In this assignment, you will perform a classification empirical study. This notebook will help you to create derived datasets in Section 2 of the assignment.

In [1]:
#let's start by installing spaCy
!pip install spacy
!python -m spacy download en_core_web_sm

zsh:1: command not found: python


In [2]:
#NLP Library
import spacy
import pandas as pd
import numpy as np
#print('Hello')

You have been given a list of datasets in the assignment description. Choose one of the datasets and provide the link below and read the dataset using pandas. You should provide a link to your own Github repository even if you are using a reduced version of a dataset from your TA's repository.

add description of the dataset and your justification of the choices made to obtain the derived datasets

In [3]:
#Load the dataset you chose.
# Make sure the Notebook can load your dataset, just like previous assignments.

#url = 'https://raw.githubusercontent.com/baharin/CSI4106-Assignment4-Datasets/main/reduced_file_cnnnews.csv'
# url = 'https://raw.githubusercontent.com/baharin/CSI4106-Assignment4-Datasets/main/reduced_drugsComTest_raw_fiveclasses.csv'
url = 'https://raw.githubusercontent.com/IshanPhadte776/CSI4106A4/main/reduced_file_AirPassengerReviews.csv'

#provide the link to the raw version of dataset. You *need* to provide a link to *your own* github repository. DO NOT use the link that is provided as an example.




In [4]:
print(url)
#data = pd.read_csv(url)
data = pd.read_csv(url, nrows=int(0.5 * pd.read_csv(url).shape[0]))

https://raw.githubusercontent.com/IshanPhadte776/CSI4106A4/main/reduced_file_AirPassengerReviews.csv


In [5]:
data.head()

,customer_review,NPS Score
0,London to Izmir via Istanbul. First time I'd ...,Passive
1,Istanbul to Bucharest. We make our check in i...,Detractor
2,Rome to Prishtina via Istanbul. I flew with t...,Detractor
3,Flew on Turkish Airlines IAD-IST-KHI and retu...,Promoter
4,Mumbai to Dublin via Istanbul. Never book Tur...,Detractor


This is where you create the NLP pipeline. load() will download the correct model (English).

In [6]:
#Loads the English-Trained Model
nlp = spacy.load("en_core_web_sm")

Applying the pipeline to every sentences creates a Document where every word is a Token object.

Doc: https://spacy.io/api/doc

Token: https://spacy.io/api/token

In [ ]:
#Apply nlp pipeline to the column that has your sentences.
data['tokenized'] = data['customer_review'].apply(nlp)

In [ ]:
data.head()

A Token object has many attributes such as part-of-speech (pos_), lemma (lemma_), etc. Take a look at the documentation to see all attributes.

The following function is an example on how you can fetch a specific pos tagging from a sentence. We return the lemmatization because we only want the infinitive word.

In [16]:
#create empty dataframes that will store your derived datasets


#Creates with two columns, Class which is blank and pos
derived_dataset1 = pd.DataFrame(columns = ['Class', 'pos'])
derived_dataset2 = pd.DataFrame(columns = ['Class', 'pos-np'])

In [17]:
def get_pos(sentence, wanted_pos): #wanted_pos refers to the desired pos tagging
    verbs = []
    for token in sentence:
        if token.pos_ in wanted_pos:
            verbs.append(token.lemma_) # lemma returns a number. lemma_ return a string
    return ' '.join(verbs) # return value is as a string and not a list for countVectorizer

In [18]:
def extract_entities_and_pos(doc, entity_types=['LOC'], pos_tags=['NOUN', 'ADJ']):
    entities = [ent.text for ent in doc.ents if ent.label_ in entity_types]
    selected_pos = [token.text for token in doc if token.pos_ in pos_tags]

    # Combine entities and selected POS into a single list or string
    result = entities + selected_pos
    return result

In [19]:
#As an example, we use the above function to fetch all the verbs. We store this information in our first derived dataset
derived_dataset1['Class'] = data['NPS Score']
derived_dataset1['pos'] = data['tokenized'].apply(lambda sent : get_pos(sent, ['ADJ']))

In [20]:
derived_dataset1.head()

,Class,pos
0,Passive,first good nice great Most contradictory littl...
1,Detractor,first last
2,Detractor,several past bad bad normal most useless few w...
3,Promoter,excellent inflight extensive easy excellent in...
4,Detractor,turkish other more


In [21]:
#Change this line to fetch your desired pos taggings for the second derived dataset
#derived_dataset2['pos-np'] = data['tokenized'].apply(lambda sent : get_pos(sent, ['PRON']))
derived_dataset2['Class'] = data['NPS Score']
derived_dataset2['pos-np'] = data['tokenized'].apply(lambda doc: extract_entities_and_pos(doc, entity_types=['GPE'], pos_tags=['NOUN', 'ADJ']))

derived_dataset2.head()

,Class,pos-np
0,Passive,"[London, Istanbul, Istanbul, Ukraine, London, ..."
1,Detractor,"[Istanbul, check, airport, luggage, gate, gate..."
2,Detractor,"[Rome, Prishtina, Istanbul, Rome, Prishtina, I..."
3,Promoter,"[quality, flights, time, catering, excellent, ..."
4,Detractor,"[Mumbai, Dublin, Istanbul, Dublin, Mumbai, Mum..."


In [22]:
#For Derived Dataset 2, you also need to include Named Entities
#Below is just an example of obtaining such entities on a specific sentence, but you would do NER
#on the dataset of your choice.
#You can choose the types of entities (dates, organization, people) that you want,
#and then in your derived dataset, just make sure you include these entities separated by spaces (as shown for verbs)
#in a previous cell.

sentence = "apple is looking at buying U.K. startup for $1 billion"
doc = nlp(sentence)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


Now that you have your derived datasets, you can move to perform your classificaton task.

**Perform A Classification Empirical Study**

1. Encode the text as input features with associated values

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#derived_dataset2['pos-np'] = derived_dataset2['pos-np'].apply(lambda x: ' '.join(x))

# Step 1: Create a CountVectorizer with optional stop words
vectorizer = CountVectorizer(stop_words='english')

#X_features_tf = vectorizer.fit_transform(derived_dataset2['pos-np'])


# Original Dataset
x = vectorizer.fit_transform(data['tokenized'].apply(lambda x: ' '.join([token.text for token in x])))
y = data['NPS Score']
# Derived-Dataset-1
x_derived1 = vectorizer.transform(derived_dataset1['pos'].apply(lambda x: ' '.join(x.split())))
y_derived1 = derived_dataset1['Class']
# Derived-Dataset-2
x_derived2 = vectorizer.transform(derived_dataset2['pos-np'].apply(lambda x: ' '.join(map(str, x))))
y_derived2 = derived_dataset2['Class']


#print(X_features_tf)
#print(x_encoded)



2. Define 2 models using some default parameters
3. Train/test/evaluate your 2 models(that have default parameters) on your 3 datasets. For each model

In [24]:
#Case 1

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import KFold
import numpy as np

# Create a Logistic Regression model with specific parameters
logreg_model = LogisticRegression(max_iter=500)

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    logreg_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = logreg_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")


Precision (Micro): 0.7230
Recall (Micro): 0.7230
Precision (Macro): 0.6686
Recall (Macro): 0.6669


In [33]:
#Case 2

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import KFold
import numpy as np

# Create a Logistic Regression model with specific parameters
logreg_model = LogisticRegression(max_iter=500)

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x_derived1):
    X_train, X_test = x_derived1[train_index], x_derived1[test_index]
    y_train, y_test = y_derived1.iloc[train_index], y_derived1.iloc[test_index]

    # Train the model
    logreg_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = logreg_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")


Precision (Micro): 0.7123
Recall (Micro): 0.7123
Precision (Macro): 0.6485
Recall (Macro): 0.6446


In [34]:
#Case 3

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import KFold
import numpy as np

# Create a Logistic Regression model with specific parameters
logreg_model = LogisticRegression(max_iter=500)

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x_derived2):
    X_train, X_test = x_derived2[train_index], x_derived2[test_index]
    y_train, y_test = y_derived2.iloc[train_index], y_derived2.iloc[test_index]

    # Train the model
    logreg_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = logreg_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")


Precision (Micro): 0.7006
Recall (Micro): 0.7006
Precision (Macro): 0.6423
Recall (Macro): 0.6414


In [35]:
# Create an MLP model with specific parameters
#Case 4
mlp_model = MLPClassifier(max_iter=500)

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    mlp_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = mlp_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")

Precision (Micro): 0.7223
Recall (Micro): 0.7223
Precision (Macro): 0.6623
Recall (Macro): 0.6591


In [ ]:
#Case 5

from sklearn.neural_network import MLPClassifier

# Create an MLP model with specific parameters
mlp_model = MLPClassifier(max_iter=500)

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x_derived1):
    X_train, X_test = x_derived1[train_index], x_derived1[test_index]
    y_train, y_test = y_derived1.iloc[train_index], y_derived1.iloc[test_index]

    # Train the model
    mlp_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = mlp_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")

Precision (Micro): 0.7323
Recall (Micro): 0.7323
Precision (Macro): 0.6251
Recall (Macro): 0.6276


In [ ]:
#Case 6

from sklearn.neural_network import MLPClassifier

# Create an MLP model with specific parameters
mlp_model = MLPClassifier(max_iter=500)

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x_derived2):
    X_train, X_test = x_derived2[train_index], x_derived2[test_index]
    y_train, y_test = y_derived2.iloc[train_index], y_derived2.iloc[test_index]

    # Train the model
    mlp_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = mlp_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")

Precision (Micro): 0.7751
Recall (Micro): 0.7751
Precision (Macro): 0.6654
Recall (Macro): 0.6506


4. Modify some parameters of the MLP model and perform a train/test/evaluate again. Do this
for two times.

5. Analyze the obtained results


In [ ]:
#Case 7

from sklearn.neural_network import MLPClassifier


# Create an MLP model with specific parameters
mod_mlp_model = MLPClassifier(max_iter=500, hidden_layer_sizes=(100,))

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    mod_mlp_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = mod_mlp_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")

Precision (Micro): 0.7788
Recall (Micro): 0.7788
Precision (Macro): 0.6499
Recall (Macro): 0.6431


In [ ]:
#Case 8

from sklearn.neural_network import MLPClassifier

# Create an MLP model with specific parameters
mod_mlp_model = MLPClassifier(max_iter=500, hidden_layer_sizes=(100,))

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x_derived1):
    X_train, X_test = x_derived1[train_index], x_derived1[test_index]
    y_train, y_test = y_derived1.iloc[train_index], y_derived1.iloc[test_index]

    # Train the model
    mod_mlp_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = mod_mlp_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")

Precision (Micro): 0.7258
Recall (Micro): 0.7258
Precision (Macro): 0.6183
Recall (Macro): 0.6150


In [32]:
#Case 9

from sklearn.neural_network import MLPClassifier

# Create an MLP model with specific parameters
mod_mlp_model = MLPClassifier(max_iter=500, hidden_layer_sizes=(100,))

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x_derived2):
    X_train, X_test = x_derived2[train_index], x_derived2[test_index]
    y_train, y_test = y_derived2.iloc[train_index], y_derived2.iloc[test_index]

    # Train the model
    mod_mlp_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = mod_mlp_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")

Precision (Micro): 0.6859
Recall (Micro): 0.6859
Precision (Macro): 0.6287
Recall (Macro): 0.6285


In [29]:
#Case 10

from sklearn.neural_network import MLPClassifier

# Create an MLP model with specific parameters
mod_mlp_model = MLPClassifier(max_iter=500, activation='tanh')

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    mod_mlp_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = mod_mlp_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Precision (Micro): 0.7203
Recall (Micro): 0.7203
Precision (Macro): 0.6564
Recall (Macro): 0.6549


In [30]:
#Case 11

from sklearn.neural_network import MLPClassifier

# Create an MLP model with specific parameters
mod_mlp_model = MLPClassifier(max_iter=500, activation='relu')

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x_derived1):
    X_train, X_test = x_derived1[train_index], x_derived1[test_index]
    y_train, y_test = y_derived1.iloc[train_index], y_derived1.iloc[test_index]

    # Train the model
    mod_mlp_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = mod_mlp_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Precision (Micro): 0.6212
Recall (Micro): 0.6212
Precision (Macro): 0.6484
Recall (Macro): 0.5499


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [31]:
from sklearn.neural_network import MLPClassifier

# Case 12
#Create an MLP model with specific parameters
mod_mlp_model = MLPClassifier(max_iter=500, activation='relu')

# Define the number of splits for KFold
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store precision and recall values for each fold
precision_micro = []
recall_micro = []
precision_macro = []
recall_macro = []

# Iterate through folds
for train_index, test_index in kf.split(x_derived2):
    X_train, X_test = x_derived2[train_index], x_derived2[test_index]
    y_train, y_test = y_derived2.iloc[train_index], y_derived2.iloc[test_index]

    # Train the model
    mod_mlp_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = mod_mlp_model.predict(X_test)

    # Calculate precision and recall for each fold
    precision_micro.append(precision_score(y_test, y_pred, average='micro', zero_division=1))
    recall_micro.append(recall_score(y_test, y_pred, average='micro'))
    precision_macro.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall_macro.append(recall_score(y_test, y_pred, average='macro'))

# Calculate average precision and recall across folds
avg_precision_micro = np.mean(precision_micro)
avg_recall_micro = np.mean(recall_micro)
avg_precision_macro = np.mean(precision_macro)
avg_recall_macro = np.mean(recall_macro)

# Print the results
print(f"Precision (Micro): {avg_precision_micro:.4f}")
print(f"Recall (Micro): {avg_recall_micro:.4f}")
print(f"Precision (Macro): {avg_precision_macro:.4f}")
print(f"Recall (Macro): {avg_recall_macro:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Precision (Micro): 0.7100
Recall (Micro): 0.7100
Precision (Macro): 0.6537
Recall (Macro): 0.6481


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Analyze the Results



1.   Logistic Regression on Original Dataset

Precision (Micro): 0.7230
Recall (Micro): 0.7230
Precision (Macro): 0.6686
Recall (Macro): 0.6669

2.   Logistic Regression on Dervied Dataset 1

Precision (Micro): 0.7123
Recall (Micro): 0.7123
Precision (Macro): 0.6485
Recall (Macro): 0.6446

3.   Logistic Regression on Dervied Dataset 2

Precision (Micro): 0.7006
Recall (Micro): 0.7006
Precision (Macro): 0.6423
Recall (Macro): 0.6414

4.   MLP Base on Original Dataset

Precision (Micro): 0.7223
Recall (Micro): 0.7223
Precision (Macro): 0.6623
Recall (Macro): 0.6591


5.   MLP Base on Dervied Dataset 1

Precision (Micro): 0.7323
Recall (Micro): 0.7323
Precision (Macro): 0.6251
Recall (Macro): 0.6276

6.   MLP Base on Dervied Dataset 2

Precision (Micro): 0.7751
Recall (Micro): 0.7751
Precision (Macro): 0.6654
Recall (Macro): 0.6506

7.   MLP HLS on Original Dataset

Precision (Micro): 0.7788
Recall (Micro): 0.7788
Precision (Macro): 0.6499
Recall (Macro): 0.6431


8.   MLP HLS on Dervied Dataset 1

Precision (Micro): 0.7258
Recall (Micro): 0.7258
Precision (Macro): 0.6183
Recall (Macro): 0.6150

9.   MLP HLS on Dervied Dataset 2

Precision (Micro): 0.6859
Recall (Micro): 0.6859
Precision (Macro): 0.6287
Recall (Macro): 0.6285

10.  MLP Activation on Original Dataset

Precision (Micro): 0.7203
Recall (Micro): 0.7203
Precision (Macro): 0.6564
Recall (Macro): 0.6549


11.  MLP Activation on Dervied Dataset 1

Precision (Micro): 0.6212
Recall (Micro): 0.6212
Precision (Macro): 0.6484
Recall (Macro): 0.5499

12.  MLP Activation on Dervied Dataset 2

Precision (Micro): 0.7100
Recall (Micro): 0.7100
Precision (Macro): 0.6537
Recall (Macro): 0.6481


For Logistic Regression, The models performs better on the original dataset compared to the deprived data set

For the Base MLP Set, the model performs best on the Deprived Data Set 2

For the HLS Model, the model performs best on the original model

For the Activitation Model, the model performs best on the original and DD2 and worst on DD1.

Overall, Dataset 1 was ineffective and Data Set 2 and the original data was helpful.

The MLP HLS on the original dataset was the best.